In [2]:
from sklearn.datasets import load_breast_cancer

In [3]:
cancer = load_breast_cancer()

In [4]:
from numpy.random import randint
from numpy.random import rand
 
# objective function
def objective(x):
    out = x[0]**2.0 + x[1]**2.0
    return out
 
# decode bitstring to numbers
def decode(bounds, n_bits, bitstring):
    decoded = list()
    largest = 2**n_bits
    for i in range(len(bounds)):
        # extract the substring
        start, end = i * n_bits, (i * n_bits)+n_bits
        substring = bitstring[start:end]
        # convert bitstring to a string of chars
        chars = ''.join([str(s) for s in substring])
        # convert string to integer
        integer = int(chars, 2)
        # scale integer to desired range
        value = bounds[i][0] + (integer/largest) * (bounds[i][1] - bounds[i][0])
        # store
        decoded.append(value)
    return decoded
 
# tournament selection
def selection(pop, scores, k=3):
    # first random selection
    selection_ix = randint(len(pop))
    for ix in randint(0, len(pop), k-1):
        # check if better (e.g. perform a tournament)
        if scores[ix] < scores[selection_ix]:
            selection_ix = ix
    return pop[selection_ix]

# crossover two parents to create two children
def crossover(p1, p2, r_cross):
    # children are copies of parents by default
    c1, c2 = p1.copy(), p2.copy()
    # check for recombination
    if rand() < r_cross:
        # select crossover point that is not on the end of the string
        pt = randint(1, len(p1)-2)
        # perform crossover
        c1 = p1[:pt] + p2[pt:]
        c2 = p2[:pt] + p1[pt:]
    return [c1, c2]

# mutation operator
def mutation(bitstring, r_mut):
    for i in range(len(bitstring)):
        # check for a mutation
        if rand() < r_mut:
            # flip the bit
            bitstring[i] = 1 - bitstring[i]


In [5]:
# genetic algorithm
def genetic_algorithm(objective, bounds, n_bits, n_iter, n_pop, r_cross, r_mut):
    # initial population of random bitstring
    pop = [randint(0, 2, n_bits*len(bounds)).tolist() for _ in range(n_pop)]
    # keep track of best solution
    best, best_eval = 0, objective(decode(bounds, n_bits, pop[0]))
    # enumerate generations
                                   
    for gen in range(n_iter):
         # decode population
        decoded = [decode(bounds, n_bits, p) for p in pop]
        # evaluate all candidates in the population
        scores = [objective(d) for d in decoded]
        # check for new best solution
        for i in range(n_pop):
            if scores[i] < best_eval:
                best, best_eval = pop[i], scores[i]
                print(">%d, new best f(%s) = %f" % (gen,  decoded[i], scores[i]))
        # select parents
        selected = [selection(pop, scores) for _ in range(n_pop)]
        # create the next generation
        children = list()
        for i in range(0, n_pop, 2):
            # get selected parents in pairs
            p1, p2 = selected[i], selected[i+1]
            # crossover and mutation
            for c in crossover(p1, p2, r_cross):
                # mutation
                mutation(c, r_mut)
                # store for next generation
                children.append(c)
        # replace population
        pop = children
    return [best, best_eval]
       

In [6]:
# define range for input
bounds = [[-5.0, 5.0], [-5.0, 5.0]]

# define the total iterations
n_iter = 100

# bits per variable
n_bits = 16

# define the population size
n_pop = 100

# crossover rate
r_cross = 0.9

# mutation rate
r_mut = 1.0 / (float(n_bits) * len(bounds)) #0.03125
# perform the genetic algorithm search


best, score = genetic_algorithm(objective, bounds, n_bits, n_iter, n_pop, r_cross, r_mut)
print('Done!')
decoded = decode(bounds, n_bits, best)
print('f(%s) = %f' % (decoded, score))
        

>0, new best f([-1.962890625, 3.40911865234375]) = 15.475030
>0, new best f([-0.339202880859375, -0.076446533203125]) = 0.120903
>1, new best f([-0.243072509765625, -0.167083740234375]) = 0.087001
>2, new best f([-0.097503662109375, -0.0341796875]) = 0.010675
>4, new best f([-0.057373046875, -0.0341796875]) = 0.004460
>5, new best f([-0.026702880859375, -0.0067138671875]) = 0.000758
>11, new best f([-0.01678466796875, -0.014495849609375]) = 0.000492
>12, new best f([-0.01068115234375, -0.014495849609375]) = 0.000324
>13, new best f([-0.01068115234375, -0.013427734375]) = 0.000294
>14, new best f([-0.00946044921875, -0.010528564453125]) = 0.000200
>15, new best f([-0.00946044921875, -0.002899169921875]) = 0.000098
>16, new best f([-0.004425048828125, -0.003509521484375]) = 0.000032
>17, new best f([-0.00213623046875, -0.0030517578125]) = 0.000014
>20, new best f([-0.001220703125, -0.0030517578125]) = 0.000011
>22, new best f([-0.00091552734375, -0.0030517578125]) = 0.000010
>22, new bes

In [7]:
# testing over breast cancer data
# define range for input
bounds = cancer.data

# define the total iterations
n_iter = 100

# bits per variable
n_bits = 16

# define the population size
n_pop = 100

# crossover rate
r_cross = 0.9

# mutation rate
r_mut = 1.0 / (float(n_bits) * len(bounds)) #0.00010984182776801405
# perform the genetic algorithm search
best, score = genetic_algorithm(objective, bounds, n_bits, n_iter, n_pop, r_cross, r_mut)
print('Done!')
decoded = decode(bounds, n_bits, best)
print('f(%s) = %f' % (decoded, score))

>0, new best f([11.25670135498047, 20.01287109375, 21.038670654296876, 11.85544921875, 17.895237579345704, 14.301383209228515, 19.598949127197265, 14.256363525390626, 21.459588012695313, 21.996678466796872, 22.305771484375, 17.653938598632813, 22.696739349365235, 22.38799133300781, 18.54207763671875, 18.557074584960937, 14.917755584716797, 18.06591613769531, 19.917330932617187, 13.8040576171875, 14.68101089477539, 10.493045288085938, 14.7893212890625, 22.311763916015625, 18.410758666992187, 16.818395385742186, 20.14447525024414, 19.01449462890625, 22.002397003173826, 15.146168823242188, 19.527703857421873, 16.446236267089844, 22.727144775390627, 23.242436523437497, 16.357027893066405, 17.698588409423827, 17.195780639648437, 16.79303192138672, 16.30691177368164, 18.86045654296875, 13.857706604003905, 18.660498046875002, 24.197602539062498, 19.788560180664064, 14.305634765625, 18.27844009399414, 13.374802734374999, 16.40321594238281, 13.540460205078126, 15.386806335449219, 18.11434936523

>6, new best f([10.558475494384766, 18.040147705078123, 20.872020874023438, 16.19162109375, 14.40881866455078, 13.917151641845702, 18.726900939941405, 18.729938964843747, 13.483285827636719, 16.371006469726563, 21.70392120361328, 17.205157928466797, 24.392972717285158, 17.199176025390624, 16.025472412109377, 19.06687072753906, 15.708028869628906, 18.764634399414064, 21.712428283691406, 13.66083038330078, 15.195204467773438, 10.079677490234374, 14.694580688476563, 21.885826416015625, 20.069896240234375, 17.11584747314453, 21.10357864379883, 18.892325439453124, 25.20580108642578, 16.29173522949219, 20.914947509765625, 12.89920928955078, 22.272816162109375, 21.47550537109375, 16.990099792480468, 20.436112060546876, 17.88834228515625, 17.416365814208987, 25.06041015625, 14.040333557128907, 18.27275238037109, 12.36790771484375, 20.45542663574219, 18.544739990234376, 19.964033203125, 18.337992858886718, 12.695665222167968, 17.932199707031252, 14.39029022216797, 15.271599884033204, 13.4140136

>23, new best f([10.499022369384766, 18.040147705078123, 20.872020874023438, 16.19162109375, 14.40881866455078, 13.917151641845702, 18.726900939941405, 18.729938964843747, 13.483285827636719, 16.371006469726563, 21.70392120361328, 17.205157928466797, 24.392972717285158, 17.325738525390626, 16.025472412109377, 19.06687072753906, 15.708028869628906, 18.764634399414064, 21.712428283691406, 13.66083038330078, 15.195204467773438, 10.079677490234374, 14.694580688476563, 21.885826416015625, 20.069896240234375, 17.11584747314453, 21.10357864379883, 18.892325439453124, 25.20580108642578, 16.29173522949219, 20.914947509765625, 12.89920928955078, 22.272816162109375, 21.47550537109375, 16.990099792480468, 20.436112060546876, 17.88834228515625, 17.416365814208987, 25.06041015625, 14.040333557128907, 18.27275238037109, 12.36790771484375, 20.45542663574219, 18.544739990234376, 19.964033203125, 18.337992858886718, 12.695665222167968, 17.932199707031252, 14.39029022216797, 15.271599884033204, 13.414013

>39, new best f([10.499022369384766, 18.039464111328126, 20.872020874023438, 16.19162109375, 14.40881866455078, 13.917151641845702, 18.726900939941405, 18.729938964843747, 13.483285827636719, 16.371006469726563, 21.70392120361328, 17.205157928466797, 24.392972717285158, 17.325738525390626, 16.025472412109377, 19.06687072753906, 15.708028869628906, 18.764634399414064, 21.712428283691406, 13.66083038330078, 15.195204467773438, 10.079677490234374, 14.694580688476563, 21.885826416015625, 20.069896240234375, 17.11584747314453, 21.10357864379883, 18.892325439453124, 25.20580108642578, 16.29173522949219, 20.914947509765625, 12.89920928955078, 22.272816162109375, 21.47550537109375, 16.990099792480468, 20.436112060546876, 17.88834228515625, 17.416365814208987, 25.06041015625, 14.040333557128907, 18.27275238037109, 12.36790771484375, 20.45542663574219, 18.544739990234376, 19.964033203125, 18.337992858886718, 12.695665222167968, 17.932199707031252, 14.39029022216797, 15.271599884033204, 13.414013

>54, new best f([10.499022369384766, 18.038096923828125, 20.872020874023438, 16.19162109375, 14.40881866455078, 13.917151641845702, 18.726900939941405, 18.729938964843747, 13.483285827636719, 16.371006469726563, 22.60642120361328, 17.205157928466797, 24.392972717285158, 17.325738525390626, 16.025472412109377, 19.06687072753906, 15.708028869628906, 18.764634399414064, 21.712428283691406, 13.66083038330078, 15.195204467773438, 10.079677490234374, 14.694580688476563, 21.885826416015625, 20.069896240234375, 17.11584747314453, 21.10357864379883, 18.892325439453124, 25.20580108642578, 16.29173522949219, 20.914947509765625, 12.89920928955078, 22.272816162109375, 21.47550537109375, 16.990099792480468, 20.436112060546876, 17.88834228515625, 17.416365814208987, 25.06041015625, 14.040333557128907, 18.27275238037109, 12.36790771484375, 20.45542663574219, 18.544739990234376, 19.964033203125, 18.337992858886718, 12.695665222167968, 17.932199707031252, 14.39029022216797, 15.271599884033204, 13.414013

>65, new best f([10.499022369384766, 18.032628173828126, 20.865927124023436, 16.19162109375, 14.40881866455078, 13.917151641845702, 18.726900939941405, 18.729938964843747, 13.483285827636719, 16.371006469726563, 22.60642120361328, 17.205157928466797, 24.392972717285158, 17.325738525390626, 16.025472412109377, 19.06687072753906, 15.708028869628906, 18.764634399414064, 21.712428283691406, 13.66083038330078, 15.195204467773438, 10.079677490234374, 14.694580688476563, 21.885826416015625, 20.069896240234375, 17.11584747314453, 21.10357864379883, 18.892325439453124, 25.20580108642578, 16.29173522949219, 20.914947509765625, 12.89920928955078, 22.272816162109375, 21.47550537109375, 16.990099792480468, 20.436112060546876, 17.88834228515625, 17.416365814208987, 25.06041015625, 14.040333557128907, 18.27275238037109, 12.36790771484375, 20.45542663574219, 18.544739990234376, 19.964033203125, 18.337992858886718, 12.695665222167968, 17.932199707031252, 14.39029022216797, 15.271599884033204, 13.414013

>73, new best f([10.499022369384766, 17.857628173828125, 20.865927124023436, 16.19162109375, 14.40881866455078, 13.917151641845702, 18.726900939941405, 18.729938964843747, 13.483285827636719, 16.371006469726563, 22.60642120361328, 17.205157928466797, 24.392972717285158, 17.325738525390626, 16.025472412109377, 19.06687072753906, 15.708028869628906, 18.764634399414064, 21.712428283691406, 13.66083038330078, 15.195204467773438, 10.079677490234374, 14.694580688476563, 21.885826416015625, 20.069896240234375, 17.11584747314453, 21.10357864379883, 18.892325439453124, 25.20580108642578, 16.29173522949219, 20.914947509765625, 12.89920928955078, 22.272816162109375, 21.47550537109375, 16.990099792480468, 20.436112060546876, 17.88834228515625, 17.416365814208987, 25.06041015625, 14.040333557128907, 18.27275238037109, 12.36790771484375, 20.45542663574219, 18.544739990234376, 19.964033203125, 18.337992858886718, 12.695665222167968, 17.932199707031252, 14.39029022216797, 15.271599884033204, 13.414013

Done!
f([10.499022369384766, 17.857628173828125, 20.865927124023436, 16.19162109375, 14.40881866455078, 13.917151641845702, 18.726900939941405, 18.729938964843747, 13.483285827636719, 16.371006469726563, 22.60642120361328, 17.205157928466797, 24.392972717285158, 17.325738525390626, 16.025472412109377, 19.06687072753906, 15.708028869628906, 18.764634399414064, 21.712428283691406, 13.66083038330078, 15.195204467773438, 10.079677490234374, 14.694580688476563, 21.885826416015625, 20.069896240234375, 17.11584747314453, 21.10357864379883, 18.892325439453124, 25.20580108642578, 16.29173522949219, 20.914947509765625, 12.89920928955078, 22.272816162109375, 21.47550537109375, 16.990099792480468, 20.436112060546876, 17.88834228515625, 17.416365814208987, 25.06041015625, 14.040333557128907, 18.27275238037109, 12.36790771484375, 20.45542663574219, 18.544739990234376, 19.964033203125, 18.337992858886718, 12.695665222167968, 17.932199707031252, 14.39029022216797, 15.271599884033204, 13.414013671875, 

In [21]:
print(decoded[len(decoded)-30:-1])

[7.892496215820312, 11.710806884765624, 18.13055419921875, 21.322762451171876, 13.872784576416016, 19.79924789428711, 16.121773223876954, 12.555303039550783, 12.5486083984375, 14.004012390136719, 22.12884552001953, 11.142450256347656, 21.853928833007814, 29.349923400878907, 20.273135147094727, 21.18978515625, 22.062716064453124, 13.46847152709961, 25.249843704223636, 15.753826293945313, 18.68822784423828, 26.600701904296873, 19.289110107421877, 25.526927490234378, 23.557491912841797, 21.66105239868164, 23.302990112304688, 27.008829956054687, 27.997108459472656]


In [9]:
print(best)

[1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 

In [10]:
print(r_mut)

0.00010984182776801405


In [11]:
print(bounds)

[[1.799e+01 1.038e+01 1.228e+02 ... 2.654e-01 4.601e-01 1.189e-01]
 [2.057e+01 1.777e+01 1.329e+02 ... 1.860e-01 2.750e-01 8.902e-02]
 [1.969e+01 2.125e+01 1.300e+02 ... 2.430e-01 3.613e-01 8.758e-02]
 ...
 [1.660e+01 2.808e+01 1.083e+02 ... 1.418e-01 2.218e-01 7.820e-02]
 [2.060e+01 2.933e+01 1.401e+02 ... 2.650e-01 4.087e-01 1.240e-01]
 [7.760e+00 2.454e+01 4.792e+01 ... 0.000e+00 2.871e-01 7.039e-02]]


In [12]:
print(len(bounds))
print(len(bounds[1]))

569
30


In [14]:
print(cancer.feature_names)
print(len(cancer.feature_names))

['mean radius' 'mean texture' 'mean perimeter' 'mean area'
 'mean smoothness' 'mean compactness' 'mean concavity'
 'mean concave points' 'mean symmetry' 'mean fractal dimension'
 'radius error' 'texture error' 'perimeter error' 'area error'
 'smoothness error' 'compactness error' 'concavity error'
 'concave points error' 'symmetry error' 'fractal dimension error'
 'worst radius' 'worst texture' 'worst perimeter' 'worst area'
 'worst smoothness' 'worst compactness' 'worst concavity'
 'worst concave points' 'worst symmetry' 'worst fractal dimension']
30
